In [1]:
import pandas as pd
import sqlite3
import math

In [2]:
# Establish a connection to the database (it will create the database file if it does not exist)
conn = sqlite3.connect('hlp_ranking_experiment.db')
# Create a cursor object using the cursor() method
c = conn.cursor()

# Queries

## Get decision_making_professional table
If company_size ends in the digit 1 it is because in the survey the respondent put the character '> - greater than' before the number. For example, the value 7001 in the company_size column was '>7000' in the survey.

In [3]:
sql_query = """
SELECT * FROM decision_making_professional
"""

In [4]:
df = pd.read_sql_query(sql_query, conn)
df

,id,position,age,gender,last_grade_completed,degree_name,years_expirience,years_expirience_hiring,company_size,profile_id
0,1,Tech Recruiter,35,Male,Bachelor's degree,Administrative Computer Science,12,12,7001,1
1,2,Principal Developer,39,Male,Master's degree,Computer Science,15,15,2000,2
2,3,CEO,44,Male,Bachelor's degree,Computer Science Engineer,21,18,15,1
3,4,Tech Recruiter,37,Female,Bachelor's degree,Business Administration,14,14,6000,1
4,5,Manager,28,Male,Master's degree,Data Science,7,5,5000,2
5,6,Tech Recruiter,26,Male,Bachelor's degree,Software Engineering,7,2,700,1
6,7,Manager,34,Male,Bachelor's degree,Information Systems Engineering,9,2,80,1
7,8,Tech Lead,28,Female,Bachelor's degree,Computer Science,8,1,25,4
8,9,Tech Lead,43,Female,Master's degree,Information Technology,20,20,200,2
9,10,Executive,52,Male,PhD,Computer Science,30,30,300,1


## Get the profiles table

In [5]:
# SQL query to get all related subjects for a given case_id
sql_query = """
SELECT * FROM profile
"""

In [6]:
df = pd.read_sql_query(sql_query, conn)
df

,id,description
0,1,"front-end, back-end, and full-stack areas"
1,2,"data science, data engineering, and machine le..."
2,3,embedded software
3,4,qa


## Obtain experimental data for a given profile_id - case_id pair

In [7]:
# The IDs you want to search for
profile_id = 1  # Profile 1 is for front-end, back-end, and full-stack areas
case_id = "Greece"  # The Greece Set is the experiment where all subjects are alike and from the same area

# SQL query using placeholders for parameters
sql_query = """
SELECT er.* , cs.case_id, cs.subject_id
FROM exp_rank AS er
JOIN experiment AS e ON er.experiment_id = e.id
JOIN case_subject AS cs ON e.case_subject_id = cs.id
WHERE cs.case_id = ? AND e.profile_id = ?
"""

In [8]:
# Define your query with parameters
parameters = (case_id, profile_id)
# Execute the query and fetch the result into a DataFrame
df = pd.read_sql_query(sql_query, conn, params=parameters)
df.head(12)

,id,experiment_id,dmp_id,rank,above_cutoff,case_id,subject_id
0,1,1,14,5,1,Greece,Delta
1,2,2,14,2,1,Greece,Kappa
2,3,3,14,3,1,Greece,Lambda
3,4,4,14,1,1,Greece,Sigma
4,5,5,14,4,1,Greece,Omega
5,6,6,14,6,1,Greece,Tau
6,19,1,10,5,0,Greece,Delta
7,20,2,10,3,1,Greece,Kappa
8,21,3,10,1,1,Greece,Lambda
9,22,4,10,4,1,Greece,Sigma


## Obtain experimental data for a specific pair dmp_id - case_id

In [9]:
# The IDs you want to search for
decision_making_professional_id = 14
case_id = 'Greece'

# SQL query using placeholders for parameters
sql_query = """
SELECT er.*, cs.case_id, cs.subject_id
FROM exp_rank er
JOIN experiment e ON er.experiment_id = e.id
JOIN case_subject cs ON e.case_subject_id = cs.id
JOIN decision_making_professional dmp ON er.dmp_id = dmp.id
WHERE cs.case_id = ? AND dmp.id = ?;
"""

In [10]:
# Define your query with parameters
parameters = (case_id, decision_making_professional_id)
# Execute the query and fetch the result into a DataFrame
df = pd.read_sql_query(sql_query, conn, params=parameters)
df

,id,experiment_id,dmp_id,rank,above_cutoff,case_id,subject_id
0,1,1,14,5,1,Greece,Delta
1,2,2,14,2,1,Greece,Kappa
2,3,3,14,3,1,Greece,Lambda
3,4,4,14,1,1,Greece,Sigma
4,5,5,14,4,1,Greece,Omega
5,6,6,14,6,1,Greece,Tau


## Description of the experimental subjects for a given case

In [11]:
case_id = 'Greece'

# SQL query to get all related subjects for a given case_id
sql_query = """
SELECT s.id, s.description
FROM subject s
JOIN case_subject cs ON s.id = cs.subject_id
WHERE cs.case_id = ?
"""

In [12]:
# Define your query with parameters
parameters = (case_id,)
# Execute the query and fetch the result into a DataFrame
df = pd.read_sql_query(sql_query, conn, params=parameters)
df

,id,description
0,Delta,With adequate experience for the position and ...
1,Kappa,With adequate experience for the position and ...
2,Lambda,Adequately experienced for the position but go...
3,Sigma,Overqualified for the position in terms of wor...
4,Omega,Overqualified for the position in terms of wor...
5,Tau,Lacks years of experience for the position. It...


In [13]:
# After making queries
conn.commit()
# Close the connection when done
conn.close()